# Data Collection for transfer analysis


## Imports
We will be using bs4 to parse through the html pages

In [0]:
from bs4 import BeautifulSoup
import urllib.request
import re

import pandas as pd
import numpy as np

from urllib.request import Request, urlopen
from random import randint,shuffle
import time
from time import sleep


In [0]:
!pip install user_agent
import user_agent
from user_agent import generate_user_agent, generate_navigator

## Scraping News Websites for traffic news

### Skysports
Skysports does not hold old transfer news (only 12 pages of it) on their website but they are lenient when it comes to scrappers

In [0]:
content = []
skysports = [
       "https://www.skysports.com/football/transfer-news",
       "https://www.skysports.com/football/transfer-news/more/2",
       "https://www.skysports.com/football/transfer-news/more/3",
       "https://www.skysports.com/football/transfer-news/more/4",
       "https://www.skysports.com/football/transfer-news/more/5",
       "https://www.skysports.com/football/transfer-news/more/6",
       "https://www.skysports.com/football/transfer-news/more/7",
       "https://www.skysports.com/football/transfer-news/more/8",
       "https://www.skysports.com/football/transfer-news/more/9",
       "https://www.skysports.com/football/transfer-news/more/10",
       "https://www.skysports.com/football/transfer-news/more/11",
       "https://www.skysports.com/football/transfer-news/more/12"
       ]
for url in skysports:
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    regex = re.compile('news-item')
    content_lis = soup.find_all('div', attrs={'class': regex})
    for div in content_lis:
        title = div.find('h2').getText()
        link = div.find('a')["href"]
        x = re.search("^/transfer" , link)
        if x != None :
          link = "https://www.skysports.com" + link;
        topic = div.find('strong').getText().strip()
        newspage = urllib.request.urlopen(link)
        tsoup = BeautifulSoup(newspage, 'html.parser')
        try:
          article_title = tsoup.find('h1').getText()
        except:
          article_title = ""
        try:
          article_subtitle = tsoup.find('h2').getText()
        except:
          article_subtitle = ""
        h3_array = tsoup.find_all('h3')
        article_part = []
        for h3 in h3_array : 
            article_part.append(h3.getText())
        article_part = '\n'.join(article_part)
        p_array = tsoup.find_all('p')
        article_content = []
        for h3 in p_array : 
            article_content.append(h3.getText())
        article_content = '\n'.join(article_content)
        regex = re.compile("highlight")
        try:
          date = tsoup.find("span" , attrs={"class":regex}).getText()
        except:
          date = ""
        article = {
            "url" : link ,
            "subscript" : title ,
            "topic" : topic ,
            "title" : article_title ,
            "subtitle" : article_subtitle ,
            "emphasis" : article_part ,
            "content" : article_content ,
            "date" : date
        }
        content.append(article)
df = pd.DataFrame(content)
df.to_csv("skysport_data.csv")

### Goal
Goal.com has all its articles accessible but they have a strict robots.txt so we'll have to use a user agent to scrape data
We'll also put a time delay as we are scraping a lot of information we do not want overload their servers with requests.

* **Irresponsible web scraping can get you blocked from websites** 
* **Data scraped belongs to goal.com. Any form of redistribution will have legal consequences**

In [0]:
headers = {'User-Agent': generate_user_agent()}
content = []
links = []
goal = [
        "https://www.goal.com/en-us/transfer-news"
       ]
for i in range(2,501):
  new_url = goal[0]+"/"+str(i)
  goal.append(new_url)
shuffle(goal)

In [0]:
import time
n = 0
program_starts = time.time()
for url in goal:
  print(url)
  now = time.time()
  print("{0} sites have been scraped .It has been {1} seconds".format(n,now - program_starts))
  sleep(randint(1,30))
  request = Request(url, headers=headers)
  page = urllib.request.urlopen(request)
  soup = BeautifulSoup(page, 'html.parser')
  content_lis = soup.find_all('article')
  for article in content_lis :
    regex = re.compile("article")
    link = article.find('a', attrs={'class': regex})
    if link != None:
      link = "https://www.goal.com"+link["href"]  
      links.append(link)
  n = n + 1
links = pd.DataFrame(links)
links.to_csv("goal.csv")  

In [0]:
df = pd.DataFrame()
links = pd.read_csv("goal.csv")
links = list(links["0"])

In [0]:
shuffle(links)

In [0]:
from tqdm import tqdm
headers = {'User-Agent': generate_user_agent()}
n = 0
program_starts = time.time()
for url in tqdm(links):
  try:
    n = n+1
    request = Request(url, headers=headers)
    page = urllib.request.urlopen(request)
    soup = BeautifulSoup(page, 'html.parser')
    try:
      article_title = soup.find("h1").getText()
    except:
      article_title = ""
    try:
      regex = re.compile("teaser")
      article_subtitle = soup.find("div",attrs={"class" : regex}).getText()
    except:
      article_subtitle = ""
    h3_array = soup.find_all('h2')
    article_part = []
    for h3 in h3_array : 
        article_part.append(h3.getText())
    article_part = '\n'.join(article_part)
    p_array = soup.find_all('p')
    article_content = []
    for h3 in p_array : 
        article_content.append(h3.getText())
    article_content = '\n'.join(article_content)
    regex = re.compile("tags-list__link")
    taglist = soup.find_all("a",attrs={"class" : regex})
    tags = []
    for tag in taglist:
      tags.append(tag.getText())
    topic = '+'.join(tags)
    datetag = soup.find("time")
    date = datetag.getText()
    article = {
              "url" : url ,
              "subscript" : article_title ,
              "topic" : topic ,
              "title" : article_title ,
              "subtitle" : article_subtitle ,
              "emphasis" : article_part ,
              "content" : article_content ,
              "date" : date.strip()
          }
    df = df.append(article , ignore_index=True)  
    df.to_csv("transfer_data.csv")
  except:
    n=n-1

100%|██████████| 13013/13013 [5:48:17<00:00,  1.61s/it]


In [0]:
df.shape

(12875, 8)

# Analysis of transfer reports


## Training name entity recognition

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import spacy
from spacy import displacy
import numpy

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/project stuff/transfer_data.csv")

In [0]:
df

,content,date,emphasis,subscript,subtitle,title,topic,url
0,Other English editions\nKane suffered a seriou...,1/9/20,Trending News,Dembele on Tottenham's radar following Kane su...,News that the England striker will miss at le...,Dembele on Tottenham's radar following Kane su...,Transfers + Tottenham Hotspur + Olympique Lyo...,https://www.goal.com/en-us/news/dembele-on-tot...
1,Other English editions\nCristian Tello will le...,15:14 6/30/17,Trending News,Tello to join Real Betis from Barcelona in €5m...,"After re-signing Gerard Deulofeu, the Catalan...",Tello to join Real Betis from Barcelona in €5m...,Transfers + Primera División + Tello + Real B...,https://www.goal.com/en-us/news/cristian-tello...
2,Other English editions\nPep Guardiola accepts ...,17:56 1/22/19,Trending News,'I don't want sad players here' - Guardiola ad...,Nicolas Otamendi has drifted out of favour at...,'I don't want sad players here' - Guardiola ad...,Transfers + Manchester City + Nicolás Otamend...,https://www.goal.com/en-us/news/i-dont-want-sa...
3,Other English editions\nPaul Clement has warne...,06:21 5/19/17,Trending News,Sigurdsson should remember Tottenham disappoin...,The Iceland international was unable to secur...,Sigurdsson should remember Tottenham disappoin...,Transfers + Swansea City + Tottenham Hotspur ...,https://www.goal.com/en-us/news/premier-league...
4,Other English editions\nNeymar’s father has re...,06:32 7/2/19,Trending News,Neymar's father offers update on transfer plan...,Reports suggesting that discussions with Camp...,Neymar's father offers update on transfer plan...,Transfers + Barcelona + Neymar + Primera Divi...,https://www.goal.com/en-us/news/neymars-father...
...,...,...,...,...,...,...,...,...
12870,Other English editions\nAntoine Griezmann unde...,12:03 6/19/18,Trending News\nRelated,Griezmann understands Atletico fans' nerves ov...,After Barcelona were linked with signing the ...,Griezmann understands Atletico fans' nerves ov...,Transfers + Atlético Madrid + Atlético Madrid...,https://www.goal.com/en-us/news/griezmann-unde...
12871,Other English editions\nVictor Lindelof has sp...,4/4/17,Trending News,Lindelof breaks silence over Man Utd links,The defender has long been linked with a move...,Lindelof breaks silence over Man Utd links,Benfica + Manchester United + Premier League ...,https://www.goal.com/en-us/news/lindelof-break...
12872,Other English editions\nSydney FC are believed...,12/5/17,Trending News\nRelated,The Covert Agent: Sydney FC interested in snar...,EXCLUSIVE: Goal's resident spy reveals Graham...,The Covert Agent: Sydney FC interested in snar...,Transfers + A-League + Sydney + Nick Fitzgera...,https://www.goal.com/en-us/news/the-covert-age...
12873,Other English editions\nLuka Jovic said his dr...,00:09 6/6/19,Trending News,'My dream comes true now' - Madrid recruit Jov...,The forward said his goodbyes to the German s...,'My dream comes true now' - Madrid recruit Jov...,Transfers + Real Madrid + Luka Jović + Primer...,https://www.goal.com/en-us/news/my-dream-comes...


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=21f948835a45ab2fb545dc18defd240a5700c96ec4c3af7964ccad5879528bb1
  Stored in directory: /tmp/pip-ephem-wheel-cache-fxe5nrvn/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
import en_core_web_md

In [0]:
nlp = en_core_web_md.load()
doc = nlp(u"Kylian Mbappe to force a move from Paris Saint Germain to join Real Madrid in 2021")

In [0]:
for token in doc.ents :
  print(token.text,spacy.explain(token.label_))

Kylian Mbappe People, including fictional
Paris Saint Germain Buildings, airports, highways, bridges, etc.
Real Madrid Companies, agencies, institutions, etc.
2021 Absolute or relative dates or periods


In [0]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import Request, urlopen
import re

In [16]:
!pip install user_agent
import user_agent
from user_agent import generate_user_agent, generate_navigator

  Created wheel for user-agent: filename=user_agent-0.1.9-cp36-none-any.whl size=18807 sha256=67974b92a392175b8ace7efc6c545a64c0490f3ac7c0d01bc249afb931c18465
  Stored in directory: /root/.cache/pip/wheels/92/80/3f/5d79277825042f2d4d447f594e3fc046f1e506f2b481d364b2
Successfully built user-agent


In [0]:
headers = {'User-Agent': generate_user_agent()}
url = "https://www.uefa.com/memberassociations/leaguesandcups/index.html" 
request = Request(url, headers=headers)
page = urllib.request.urlopen(request)
soup = BeautifulSoup(page, 'html.parser')
regex = re.compile('^\d+')
countries = [(x.text,"https://www.uefa.com/"+x.find("a")["href"]) for x in soup.find_all('li', attrs={'class': regex})]

In [0]:
teams = []
for country,url in countries:
  request = Request(url, headers=headers)
  page = urllib.request.urlopen(request)
  soup = BeautifulSoup(page, 'html.parser')
  regex = re.compile('^club')
  teams = teams + [(team.text , "https://www.uefa.com/"+team.find("a")["href"],country) for team in soup.find_all("td" , attrs={"class",regex})]

In [19]:
from tqdm import tqdm
players = []
for team in tqdm(teams):
  url = team[1]
  request = Request(url, headers=headers)
  page = urllib.request.urlopen(request)
  soup = BeautifulSoup(page, 'html.parser')
  regex = re.compile('^playername')
  players = players + [(x.text,team[0]) for x in soup.find_all("td",attrs={"class":regex})]

100%|██████████| 721/721 [09:34<00:00,  1.25it/s]


In [0]:
from random import shuffle
shuffle(teams) 
shuffle(players)

In [0]:
TRAINING_DATA = []

### Training the named entity recognizer of a en_core_web_md

In [0]:
LABEL1 = "TEAM"
LABEL2 = "PLAYER"

In [0]:
for team in teams[:200]:
  sentence = team[0]+" is a club from "+team[2]+"."
  ent = {"entities" : [(0,len(team[0]),LABEL1),(len(team[0])+16,len(sentence)-1,"GPE")]}
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for team in teams[200:400]:
  sentence = team[0]+" plays in "+team[2]+"."
  ent = {"entities" : [(0,len(team[0]),LABEL1),(len(team[0])+10,len(sentence)-1,"GPE")]}
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for team in teams[400:550]:
  sentence = team[0]+" is a successful club."
  ent = {"entities" : [(0,len(team[0]),LABEL1)]}
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for team in teams[550:]:
  sentence = "The "+team[2]+" based club "+team[0]+"."
  ent = {"entities" : [(4,len(team[2]),"GPE"),(16+len(team[2]),16+len(team[2])+len(team[0]),LABEL1)]}
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[0:100]:
  sentence = player[0]+" wants to join "+player[1]
  ent = {
      "entities" : [
                    (0 , len(player[0]) ,LABEL2 ) ,
                    (15 + len(player[0]), 15 + len(player[0]) + len(player[1]) , LABEL1)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[100:200]:
  sentence = player[0]+" is looking to leave "+player[1]
  ent = {
      "entities" : [
                    (0 , len(player[0]) ,LABEL2 ) ,
                    (21 + len(player[0]), 21 + len(player[0]) + len(player[1]) , LABEL1)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[200:300]:
  sentence = player[0]+" is happy at "+player[1]
  ent = {
      "entities" : [
                    (0 , len(player[0]) ,LABEL2 ) ,
                    (13 + len(player[0]), 13 + len(player[0]) + len(player[1]) , LABEL1)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[300:400]:
  sentence = player[1]+" wants to sign "+player[0]
  ent = {
      "entities" : [
                    (0 , len(player[1]) ,LABEL1 ) ,
                    (15 + len(player[1]), 15 + len(player[0]) + len(player[1]) , LABEL2)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[400:500]:
  sentence = player[1]+" makes offer for "+player[0]
  ent = {
      "entities" : [
                    (0 , len(player[1]) ,LABEL1 ) ,
                    (16 + len(player[1]), 16 + len(player[0]) + len(player[1]) , LABEL2)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])

for player in players[500:600]:
  sentence = player[1]+" trying to kick out "+player[0]
  ent = {
      "entities" : [
                    (0 , len(player[1]) ,LABEL1 ) ,
                    (20 + len(player[1]), 20 + len(player[0]) + len(player[1]) , LABEL2)
                    ]
      }
  TRAINING_DATA = TRAINING_DATA + [(sentence , ent)]
print(TRAINING_DATA[-1])



('RB Leipzig is a club from Germany.', {'entities': [(0, 10, 'TEAM'), (26, 33, 'GPE')]})
('Coleraine FC plays in Northern Ireland.', {'entities': [(0, 12, 'TEAM'), (22, 38, 'GPE')]})
('FC Etzella Ettelbruck is a successful club.', {'entities': [(0, 21, 'TEAM')]})
('The Slovakia based club SK Sered.', {'entities': [(4, 8, 'GPE'), (24, 32, 'TEAM')]})
('Anderson Esiti wants to join PAOK FC', {'entities': [(0, 14, 'PLAYER'), (29, 36, 'TEAM')]})
('Max Grün is looking to leave VfL Borussia Mönchengladbach', {'entities': [(0, 8, 'PLAYER'), (29, 57, 'TEAM')]})
('Danijel Subašić is happy at AS Monaco FC', {'entities': [(0, 15, 'PLAYER'), (28, 40, 'TEAM')]})
('Eintracht Frankfurt wants to sign Almamy Touré', {'entities': [(0, 19, 'TEAM'), (34, 46, 'PLAYER')]})
('SL Benfica makes offer for Pizzi', {'entities': [(0, 10, 'TEAM'), (26, 31, 'PLAYER')]})
('Juventus trying to kick out Mattia De Sciglio', {'entities': [(0, 8, 'TEAM'), (28, 45, 'PLAYER')]})


In [0]:
ner = nlp.get_pipe("ner")
ner.add_label(LABEL1)
ner.add_label(LABEL2)

In [0]:
move_names = list(ner.move_names)
print(move_names)
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
other_pipes

['B-ORG', 'B-DATE', 'B-PERSON', 'B-GPE', 'B-MONEY', 'B-CARDINAL', 'B-NORP', 'B-PERCENT', 'B-WORK_OF_ART', 'B-LOC', 'B-TIME', 'B-QUANTITY', 'B-FAC', 'B-EVENT', 'B-ORDINAL', 'B-PRODUCT', 'B-LAW', 'B-LANGUAGE', 'I-ORG', 'I-DATE', 'I-PERSON', 'I-GPE', 'I-MONEY', 'I-CARDINAL', 'I-NORP', 'I-PERCENT', 'I-WORK_OF_ART', 'I-LOC', 'I-TIME', 'I-QUANTITY', 'I-FAC', 'I-EVENT', 'I-ORDINAL', 'I-PRODUCT', 'I-LAW', 'I-LANGUAGE', 'L-ORG', 'L-DATE', 'L-PERSON', 'L-GPE', 'L-MONEY', 'L-CARDINAL', 'L-NORP', 'L-PERCENT', 'L-WORK_OF_ART', 'L-LOC', 'L-TIME', 'L-QUANTITY', 'L-FAC', 'L-EVENT', 'L-ORDINAL', 'L-PRODUCT', 'L-LAW', 'L-LANGUAGE', 'U-ORG', 'U-DATE', 'U-PERSON', 'U-GPE', 'U-MONEY', 'U-CARDINAL', 'U-NORP', 'U-PERCENT', 'U-WORK_OF_ART', 'U-LOC', 'U-TIME', 'U-QUANTITY', 'U-FAC', 'U-EVENT', 'U-ORDINAL', 'U-PRODUCT', 'U-LAW', 'U-LANGUAGE', 'O', 'B-TEAM', 'I-TEAM', 'L-TEAM', 'U-TEAM', 'B-PLAYER', 'I-PLAYER', 'L-PLAYER', 'U-PLAYER']


['tagger', 'parser']

In [0]:
from spacy.util import compounding , minibatch
n_iter = 10
optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes):  # only train NER
  sizes = compounding(1.0, 4.0, 1.001)
  for itn in tqdm(range(n_iter)):
      shuffle(TRAINING_DATA)
      batches = minibatch(TRAINING_DATA, size=sizes)
      losses = {}
      for batch in batches:
          texts, annotations = zip(*batch)
          nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      print("Losses", losses)

 10%|█         | 1/10 [00:35<05:18, 35.41s/it]

Losses {'ner': 6640.267859108897}


 20%|██        | 2/10 [00:54<04:03, 30.49s/it]

Losses {'ner': 5714.571954773051}


 30%|███       | 3/10 [01:09<03:00, 25.77s/it]

...

Losses {'ner': 5362.253509901296}


 90%|█████████ | 9/10 [02:51<00:18, 18.16s/it]

Losses {'ner': 5168.127331932454}


100%|██████████| 10/10 [03:09<00:00, 18.93s/it]

Losses {'ner': 5315.679766376521}


In [0]:
!pip install pathlib
from pathlib import Path
new_model_name = "en_fnew_md"
output_dir = "./en_fnew_md"
output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta["name"] = new_model_name  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to en_fnew_md


### Completely Training a large language model

We will be training the dependency parser as well as the named entity recognizer again
This time while training the NER we will try making a more natural Training dataset 

#### Training the NER (2nd approach)

In [0]:
LABEL1 = "TEAM"
LABEL2 = "PLAYER"

In [0]:
players[0]

('Jan Koprivec', 'Kilmarnock FC')

In [0]:
teams[0]

('Southampton FC',
 'https://www.uefa.com//teamsandplayers/teams/club=52923/domestic/index.html',
 'England')

In [0]:
templates = [{"sent" : x , "tags" : y} for x,y in zip( df["title"][0:100] , df["topic"][0:100] )]

In [0]:
for tem in templates:
  tags = [x.strip() for x in tem["tags"].split("+")]
  doc = nlp(tem["sent"])
  tem["gaps"] = []
  tem["label"] = []
  print(tem["sent"])
  print(tags)
  for token in doc.ents:
    prev = 0
    if token.text in tags:
      print(token.text , token.label_)
    elif token.label_ in ["PERSON","ORG","GPE"]:
      print(token.text , token.label_)
    ltype = input()
    if ltype == 'p':
      regex = re.compile(token.text)
      loc = re.search(regex , tem["sent"]).span()
      tem["label"].append(LABEL2)
      prev = loc[1]
      tem["sent"] = tem["sent"].replace(token.text , "<PLAYER>" , 1)
    elif ltype == 'o':
      regex = re.compile(token.text)
      loc = re.search(regex , tem["sent"]).span()
      tem["label"].append(LABEL1)
      prev = loc[1]
      tem["sent"] = tem["sent"].replace(token.text , "<TEAM>" , 1)

In [0]:
templates = pd.DataFrame(templates)
templates.to_csv("/content/gdrive/My Drive/project stuff/training_template.csv")

In [0]:
templates = pd.read_csv("/content/gdrive/My Drive/project stuff/training_template.csv")
template = list(templates["sent"])

In [0]:
from random import randint
def getTrainingSample():
  sent = template[randint(0,len(template)-1)]
  ent = {"entities" : [] }
  key = re.search(r"<\w+>" , sent)
  while key != None :
    if key.group() == "<PLAYER>":
      player = players[randint(0,len(players)-1)][0]
      sent = sent.replace("<PLAYER>" , player , 1)
      ent["entities"].append((key.span()[0] , key.span()[0] + len(player) , LABEL2))
    elif key.group() == "<TEAM>":
      team = teams[randint(0,len(teams)-1)][0]
      sent = sent.replace("<TEAM>" , team , 1)
      ent["entities"].append((key.span()[0] , key.span()[0] + len(team) , LABEL1))
    key = re.search(r"<\w+>" , sent)
  return (sent , ent)
print(getTrainingSample())
shuffle(TRAINING_DATA)

("'Is Azz-Eddine Ounahi leaving? Is he ill?' - Wright demands FC Girondins de Bordeaux clarify German's future", {'entities': [(4, 21, 'PLAYER'), (60, 84, 'TEAM')]})


In [0]:
for i in range(0,1000):
  TRAINING_DATA.append(getTrainingSample())

In [0]:
import nltk
nltk.download("brown")
from nltk.corpus import brown
for x in brown.fileids()[:5]:
  for y in brown.sents(x):
    TRAINING_DATA.append((
        " ".join(y).replace("`","").replace("'","") ,
        {"entities" : []}
    ))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [0]:
len(TRAINING_DATA)

2801

In [0]:
nlp = en_core_web_md.load()

In [0]:
list(df["title"])[9983]

"'Leave Real Madrid? I have the same desire to die!' - Navas bullish despite Courtois arrival"

In [32]:
from random import randint , shuffle
def test_model():
  print("\nTESTING MODEL\n")
  for i in range(0,5):
    text = list(df["title"])[randint(0,10000)]
    doc = nlp(text)
    print("\t"+text)
    print("\t\tEntities", [(ent.text, ent.label_) for ent in doc.ents])
    print("")
test_model()


TESTING MODEL

	'If I can bring Kylian to Real Madrid I will try' - Hazard would relish link-up with Mbappe
		Entities [('Kylian', 'PERSON')]

	Cardona confirms Monterrey exit as he dreams of playing for Boca
		Entities [('Cardona', 'PERSON'), ('Monterrey', 'GPE'), ('Boca', 'FAC')]

	Lille president 'confirms' Pepe to Arsenal for €80m
		Entities [('Lille', 'GPE'), ('Pepe', 'PERSON'), ('€80m', 'MONEY')]

	Sarri requests Chelsea exit as agreement reached with Juventus
		Entities [('Sarri', 'PERSON'), ('Chelsea', 'PERSON'), ('Juventus', 'ORG')]

	Inter Miami have held Cavani talks as Beckham’s new MLS franchise seek marquee signing
		Entities [('Inter Miami', 'ORG'), ('Cavani', 'PERSON'), ('Beckham', 'PERSON'), ('MLS', 'ORG')]



In [0]:
ner = nlp.get_pipe("ner")
ner.add_label(LABEL1)
ner.add_label(LABEL2)

In [0]:
move_names = list(ner.move_names)
print(move_names)
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
other_pipes

['B-ORG', 'B-DATE', 'B-PERSON', 'B-GPE', 'B-MONEY', 'B-CARDINAL', 'B-NORP', 'B-PERCENT', 'B-WORK_OF_ART', 'B-LOC', 'B-TIME', 'B-QUANTITY', 'B-FAC', 'B-EVENT', 'B-ORDINAL', 'B-PRODUCT', 'B-LAW', 'B-LANGUAGE', 'I-ORG', 'I-DATE', 'I-PERSON', 'I-GPE', 'I-MONEY', 'I-CARDINAL', 'I-NORP', 'I-PERCENT', 'I-WORK_OF_ART', 'I-LOC', 'I-TIME', 'I-QUANTITY', 'I-FAC', 'I-EVENT', 'I-ORDINAL', 'I-PRODUCT', 'I-LAW', 'I-LANGUAGE', 'L-ORG', 'L-DATE', 'L-PERSON', 'L-GPE', 'L-MONEY', 'L-CARDINAL', 'L-NORP', 'L-PERCENT', 'L-WORK_OF_ART', 'L-LOC', 'L-TIME', 'L-QUANTITY', 'L-FAC', 'L-EVENT', 'L-ORDINAL', 'L-PRODUCT', 'L-LAW', 'L-LANGUAGE', 'U-ORG', 'U-DATE', 'U-PERSON', 'U-GPE', 'U-MONEY', 'U-CARDINAL', 'U-NORP', 'U-PERCENT', 'U-WORK_OF_ART', 'U-LOC', 'U-TIME', 'U-QUANTITY', 'U-FAC', 'U-EVENT', 'U-ORDINAL', 'U-PRODUCT', 'U-LAW', 'U-LANGUAGE', 'O', 'B-TEAM', 'I-TEAM', 'L-TEAM', 'U-TEAM', 'B-PLAYER', 'I-PLAYER', 'L-PLAYER', 'U-PLAYER']


['tagger', 'parser']

In [0]:
from IPython.display import clear_output
from spacy.util import compounding , minibatch
n_iter = 30
optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes):  # only train NER
  sizes = compounding(1.0, 4.0, 1.001)
  last_loss = 42327.484182610715 
  for itn in tqdm(range(n_iter)):
      shuffle(TRAINING_DATA)
      batches = minibatch(TRAINING_DATA, size=sizes)
      losses = {}
      for batch in batches:
          texts, annotations = zip(*batch)
          nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      clear_output(wait=True)
      test_model()
      print("Loss ", losses)
      print("Rate ", (last_loss - losses["ner"])/last_loss)
      last_loss = losses["ner"]


100%|██████████| 30/30 [18:06<00:00, 36.23s/it]


TESTING MODEL

	Gullit urges Chelsea to reject Madrid advances for Hazard
		Entities []
		Tokens [('Gullit', '', 2), ('urges', '', 2), ('Chelsea', '', 2), ('to', '', 2), ('reject', '', 2), ('Madrid', '', 2), ('advances', '', 2), ('for', '', 2), ('Hazard', '', 2)]

	Juventus to sign Bernardeschi for €40m
		Entities [('Juventus', 'TEAM'), ('Bernardeschi', 'PLAYER')]
		Tokens [('Juventus', 'TEAM', 3), ('to', '', 2), ('sign', '', 2), ('Bernardeschi', 'PLAYER', 3), ('for', '', 2), ('€', '', 2), ('40', '', 2), ('m', '', 2)]

	Hazard drops hint that he'd consider Chelsea exit
		Entities []
		Tokens [('Hazard', '', 2), ('drops', '', 2), ('hint', '', 2), ('that', '', 2), ('he', '', 2), ("'d", '', 2), ('consider', '', 2), ('Chelsea', '', 2), ('exit', '', 2)]

	Good for Bayern that Aubameyang is not leaving Dortmund – Lewandowski
		Entities []
		Tokens [('Good', '', 2), ('for', '', 2), ('Bayern', '', 2), ('that', '', 2), ('Aubameyang', '', 2), ('is', '', 2), ('not', '', 2), ('leaving', '', 2), (

In [0]:
!pip install pathlib
from pathlib import Path
new_model_name = "en_fnew_md"
output_dir = "./en_fnew_md"
output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta["name"] = new_model_name  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to en_fnew_md


In [0]:
!zip -r /content/gdrive/My\ Drive/project\ stuff/fmodel.zip en_fnew_md

updating: en_fnew_md/ (stored 0%)
updating: en_fnew_md/parser/ (stored 0%)
updating: en_fnew_md/parser/model (deflated 7%)
updating: en_fnew_md/parser/moves (deflated 56%)
updating: en_fnew_md/parser/cfg (deflated 49%)
updating: en_fnew_md/meta.json (deflated 46%)
updating: en_fnew_md/tagger/ (stored 0%)
updating: en_fnew_md/tagger/model (deflated 7%)
updating: en_fnew_md/tagger/cfg (deflated 34%)
updating: en_fnew_md/tagger/tag_map (deflated 50%)
updating: en_fnew_md/ner/ (stored 0%)
updating: en_fnew_md/ner/model (deflated 8%)
updating: en_fnew_md/ner/moves (deflated 78%)
updating: en_fnew_md/ner/cfg (deflated 49%)
updating: en_fnew_md/tokenizer (deflated 79%)
updating: en_fnew_md/vocab/ (stored 0%)
updating: en_fnew_md/vocab/key2row (deflated 78%)
updating: en_fnew_md/vocab/strings.json (deflated 61%)
updating: en_fnew_md/vocab/lookups.bin (deflated 45%)
updating: en_fnew_md/vocab/vectors (deflated 9%)
updating: en_fnew_md/vocab/lexemes.bin (deflated 64%)


#### Using Matcher and phrase matcher to improve training data quality 

In [23]:
import seaborn as sns


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [24]:
players[0]

('Catalin Constantin Barbu', 'Chindia Targoviste')

In [25]:
teams[0]

('ŁKS Łódź',
 'https://www.uefa.com//teamsandplayers/teams/club=53004/domestic/index.html',
 'Poland')

In [26]:
df["title"][0]

"Dembele on Tottenham's radar following Kane surgery blow"

In [27]:
df["topic"][0]

' Transfers + Tottenham Hotspur + Olympique Lyonnais + Moussa Dembélé + Premier League + Harry Kane + Ligue 1 '

In [0]:
import spacy
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

In [29]:
import unicodedata
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text)
s = strip_accents('àéêöhello')
print(s)

aeeohello


In [0]:
status = {}

In [31]:
## enter possible extra team names
txt = input()
while txt != "":
  status[txt] = "TEAM"
  txt = input()  

liv
reds
psg
bar
fcb
man city
man utd
utd
rma
blues
whites
seagulls
wolves
cules



In [33]:
import re
from tqdm import tqdm

for title in tqdm(df["content"]):
  doc = nlp(title.lower())
  for t in doc.ents:
    if t.label_ in ["ORG" , "GPE" , "PERSON","FAC"]:
      match = re.search(r"(([a-z]+.?)+)",t.text)
      if match != None:
        t = strip_accents(match.group())
        if t not in status: 
          for team in teams:  
            if t in strip_accents(team[0].lower()):
              status[t] = "TEAM"
              break
        if t not in status:
          for player in players:
            if t in strip_accents(player[0].lower()):
              status[t] = "PLAYER"
              break
for title in tqdm(df["title"]):
  doc = nlp(title.lower())
  for t in doc.ents:
    if t.label_ in ["ORG" , "GPE" , "PERSON"]:
      match = re.search(r"(([a-z]+.?)+)",t.text)
      if match != None:
        t = strip_accents(match.group())
        if t not in status: 
          for team in teams:  
            if t in strip_accents(team[0].lower()):
              status[t] = "TEAM"
              break
        if t not in status:
          for player in players:
            if t in strip_accents(player[0].lower()):
              status[t] = "PLAYER"
              break

100%|██████████| 12580/12580 [04:18<00:00, 48.67it/s]


In [0]:
def getLabel(x):
  x = strip_accents(x.lower())
  if x in status:
    return status[x]
  else:
    return "None"

In [35]:
'''
  TRAIN_DATA = [
      ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
      ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
  ]
'''
TRAINING_DATA = []
for x in tqdm(df["title"]):
  sentence = x
  doc = nlp(x)
  entities = []
  for token in doc.ents:
    start = token.start_char-token.sent.start_char
    end = token.end_char-token.sent.start_char
    if token.label_  in ["ORG" , "GPE" , "PERSON","FAC"]:
      if getLabel(token.text) == "None" :
        label = token.label_
      else:
        label = getLabel(token.text)
    else:
      label = token.label_
    entities.append((start ,end, label))
  TRAINING_DATA.append((sentence,{"entities" : entities}))
len(TRAINING_DATA)

100%|██████████| 12580/12580 [02:05<00:00, 99.95it/s] 


12580

In [36]:
import nltk
nltk.download("brown")
from nltk.corpus import brown
for x in brown.fileids()[5:8]:
  for y in brown.sents(x):
    y = " ".join(y)
    sentence = y
    doc = nlp(y)
    entities = []
    for token in doc.ents:
      start = token.start_char-token.sent.start_char
      end = token.end_char-token.sent.start_char
      label = token.label_
      entities.append((start ,end, label))
    TRAINING_DATA.append((sentence,{"entities" : entities}))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [37]:
shuffle(TRAINING_DATA)
len(TRAINING_DATA)

12900

In [38]:
TRAINING_DATA[4]

('I will definitely be at Bayer Leverkusen next season - Havertz',
 {'entities': [(24, 40, 'ORG')]})

In [39]:
def check_validity(td):
  start = 0 
  end = -1
  for ent in td[1]["entities"]:
    start = ent[0]
    if start < end:
      return(False)
      break
    end = ent[1] 
  return(True)
TRAINING_DATA = [x for x in TRAINING_DATA if check_validity(x)]
len(TRAINING_DATA)

11861

In [0]:
ner = nlp.get_pipe("ner")
ner.add_label(LABEL1)
ner.add_label(LABEL2)

In [41]:
move_names = list(ner.move_names)
print(move_names)
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
other_pipes

['B-ORG', 'B-DATE', 'B-PERSON', 'B-GPE', 'B-MONEY', 'B-CARDINAL', 'B-NORP', 'B-PERCENT', 'B-WORK_OF_ART', 'B-LOC', 'B-TIME', 'B-QUANTITY', 'B-FAC', 'B-EVENT', 'B-ORDINAL', 'B-PRODUCT', 'B-LAW', 'B-LANGUAGE', 'I-ORG', 'I-DATE', 'I-PERSON', 'I-GPE', 'I-MONEY', 'I-CARDINAL', 'I-NORP', 'I-PERCENT', 'I-WORK_OF_ART', 'I-LOC', 'I-TIME', 'I-QUANTITY', 'I-FAC', 'I-EVENT', 'I-ORDINAL', 'I-PRODUCT', 'I-LAW', 'I-LANGUAGE', 'L-ORG', 'L-DATE', 'L-PERSON', 'L-GPE', 'L-MONEY', 'L-CARDINAL', 'L-NORP', 'L-PERCENT', 'L-WORK_OF_ART', 'L-LOC', 'L-TIME', 'L-QUANTITY', 'L-FAC', 'L-EVENT', 'L-ORDINAL', 'L-PRODUCT', 'L-LAW', 'L-LANGUAGE', 'U-ORG', 'U-DATE', 'U-PERSON', 'U-GPE', 'U-MONEY', 'U-CARDINAL', 'U-NORP', 'U-PERCENT', 'U-WORK_OF_ART', 'U-LOC', 'U-TIME', 'U-QUANTITY', 'U-FAC', 'U-EVENT', 'U-ORDINAL', 'U-PRODUCT', 'U-LAW', 'U-LANGUAGE', 'O', 'B-TEAM', 'I-TEAM', 'L-TEAM', 'U-TEAM', 'B-PLAYER', 'I-PLAYER', 'L-PLAYER', 'U-PLAYER']


['tagger', 'parser']

In [44]:
from IPython.display import clear_output
from spacy.util import compounding , minibatch
n_iter = 1
optimizer = nlp.resume_training()
with nlp.disable_pipes(*other_pipes):  # only train NER
  sizes = compounding(1.0, 4.0, 1.001)
  last_loss = pd.DataFrame([{"x":0 , "y" : 100000}])
  count = 0
  for itn in tqdm(range(n_iter)):
      shuffle(TRAINING_DATA)
      batches = minibatch(TRAINING_DATA, size=sizes)
      losses = {}
      for batch in batches:
          texts, annotations = zip(*batch)
          nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
      clear_output(wait=True)
      test_model()
      print("Loss ", losses)
      count = count + 1

100%|██████████| 1/1 [02:47<00:00, 167.03s/it]


TESTING MODEL

	Donovan 'a little worried' about Pulisic's £58m Chelsea move
		Entities [('Donovan', 'PLAYER'), ('Pulisic', 'NORP'), ('£58m', 'MONEY')]

	Forza Roma! – Schick backtracks on Man Utd hint
		Entities [('Forza Roma', 'ORG'), ('Schick', 'PLAYER'), ('Man Utd', 'TEAM')]

	Dimas, Nichols headline Wanderers departures
		Entities [('Dimas', 'PLAYER'), ('Nichols', 'PLAYER')]

	Wenger regrets never signing Carrick for Arsenal
		Entities [('Wenger', 'ORG'), ('Carrick', 'TEAM'), ('Arsenal', 'TEAM')]

	Weah expecting to play as a No. 9 with Celtic
		Entities [('Weah', 'PLAYER'), ('9', 'CARDINAL'), ('Celtic', 'NORP')]

Loss  {'ner': 80063.22681556157}


In [47]:
!pip install pathlib
from pathlib import Path
new_model_name = "en_fnew_md"
output_dir = "./en_fnew_md"
output_dir = Path(output_dir)
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta["name"] = new_model_name  # rename model
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to en_fnew_md


In [48]:
!zip -r /content/gdrive/My\ Drive/project\ stuff/fmodel.zip en_fnew_md

updating: en_fnew_md/ (stored 0%)
updating: en_fnew_md/parser/ (stored 0%)
updating: en_fnew_md/parser/model (deflated 7%)
updating: en_fnew_md/parser/moves (deflated 56%)
updating: en_fnew_md/parser/cfg (deflated 49%)
updating: en_fnew_md/meta.json (deflated 46%)
updating: en_fnew_md/tagger/ (stored 0%)
updating: en_fnew_md/tagger/model (deflated 7%)
updating: en_fnew_md/tagger/cfg (deflated 34%)
updating: en_fnew_md/tagger/tag_map (deflated 50%)
updating: en_fnew_md/ner/ (stored 0%)
updating: en_fnew_md/ner/model (deflated 9%)
updating: en_fnew_md/ner/moves (deflated 78%)
updating: en_fnew_md/ner/cfg (deflated 49%)
updating: en_fnew_md/tokenizer (deflated 79%)
updating: en_fnew_md/vocab/ (stored 0%)
updating: en_fnew_md/vocab/key2row (deflated 78%)
updating: en_fnew_md/vocab/strings.json (deflated 61%)
updating: en_fnew_md/vocab/lookups.bin (deflated 45%)
updating: en_fnew_md/vocab/vectors (deflated 9%)
updating: en_fnew_md/vocab/lexemes.bin (deflated 64%)


## Cleaning Report data

In [0]:
import pandas as pd
import spacy
from spacy import displacy
import numpy

In [0]:
!unzip /content/gdrive/My\ Drive/project\ stuff/fmodel.zip 

Archive:  /content/gdrive/My Drive/project stuff/fmodel.zip
replace en_fnew_md/parser/model? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
nlp = spacy.load("en_fnew_md")

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/project stuff/transfer_data.csv")

In [0]:
def cleantext(text):
  text=text.replace("\n"," ")
  text=text.replace("Other English editions","")
  text=text[:-193]
  text.strip()
  return(text)
df["content"] = df["content"].apply(cleantext);

In [0]:
import re
def datefix(x):
  match = re.search(r"\d+/\d+/\d+",x)
  if match == None:
    print(x)
  else:
    return(str(match.group()))
df["date"] = df["date"].apply(datefix)

In [0]:
def checkemph(x):
  doc = nlp(x.lower())
  for token in doc.ents:
    if token.label_ == "PERSON":
      return False
  return True
filtered_df = df[df["emphasis"].apply(checkemph)]

In [0]:
df = filtered_df
del filtered_df

In [0]:
df.columns

Index(['content', 'date', 'emphasis', 'subscript', 'subtitle', 'title',
       'topic', 'url'],
      dtype='object')

In [0]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

In [0]:
pattern = [{"lemma" : "sign"}]
matcher.add("Sign" , None , pattern)

for title in df["title"][0:10]:
  doc = nlp(title)
  matches = matcher(doc)
  for match_id, start, end in matches:
      matched_span = doc[start:end]
      print(title)
      print(matched_span.text)
      print()

How special video helped Dortmund beat Man Utd to Haaland as Raiola confirmed signing on Christmas Eve
signing



In [0]:
for title in df["title"][123:124]:
  doc = nlp(title)
  displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

In [0]:
doc1 = nlp(df["content"][0])
sum = 0.0
for c in df["content"][1:]:
  doc = nlp(c)
  sum = sum + doc.similarity(doc2)
  doc1 = doc

In [55]:
doc = nlp(df["title"][1])
for ent in doc.ents:
  print(ent.text , ent.label_)

Tello PLAYER
Real Betis TEAM
Barcelona TEAM
€5m MONEY
